# check metrics

Algunos recursos utiles:
- https://www.evidentlyai.com/classification-metrics/multi-class-metrics
- https://arize.com/blog-course/f1-score/#:~:text=F1%20score%20is%20a%20measure,can%20be%20modified%20into%20F0.


In [1]:
import cv2
import os
import time
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Flatten, ConvLSTM2D, Reshape, BatchNormalization, Input, TimeDistributed
from tensorflow.keras.models import load_model, Model
from concurrent.futures import ThreadPoolExecutor

2024-06-10 23:38:11.122074: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 23:38:11.176690: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 23:38:11.176726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 23:38:11.179057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 23:38:11.189232: I tensorflow/core/platform/cpu_feature_guar

## Constants

In [2]:
emociones = ['Neutral', 'Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']
# IMG_SIZE = 112
# IMG_CHANNELS = 3
MAX_SEQ_LENGTH = 10
# NUM_FEATURES = 1024
# EPOCHS = 100
BATCH_SIZE=512

NUMPY_PATH = "../Modelos/tmp/np_arr/"
MODELS_PATH = "../Modelos/"

In [12]:
def load_model_keras(model_name):
    model = load_model(MODELS_PATH + model_name)
    return model

In [13]:
RNN_MODEL_NAME = "cotatest_rnn_att_v2_binary.keras"

model = load_model_keras(RNN_MODEL_NAME)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 10, 1024)]           0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 lstm_2 (LSTM)               (None, 10, 8)                33056     ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
 additive_attention_2 (Addi  (None, 10, 8)                8         ['lstm_2[0][0]',        

In [14]:
validation_ds = tf.data.Dataset.load(f"../datasets/processed/rnn_val_{MAX_SEQ_LENGTH}_v2_binary")
validation_ds

<_LoadDataset element_spec=((TensorSpec(shape=(10, 1024), dtype=tf.float32, name=None), TensorSpec(shape=(10,), dtype=tf.bool, name=None)), TensorSpec(shape=(10, 1), dtype=tf.float32, name=None))>

In [15]:
for train_data, labels in validation_ds.take(1):
    print(train_data[0].shape)
    print(train_data[1].shape)
    print(labels.shape)
    break

(10, 1024)
(10,)
(10, 1)


In [16]:
BATCH_SIZE = 512
validation_ds_batch = validation_ds.batch(BATCH_SIZE)
validation_ds_batch

<_BatchDataset element_spec=((TensorSpec(shape=(None, 10, 1024), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.bool, name=None)), TensorSpec(shape=(None, 10, 1), dtype=tf.float32, name=None))>

In [17]:
for validation_data, labels in validation_ds.take(1):
    print(validation_data[0])
    print(validation_data[1])
    print(labels)
    break

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10, 1024), dtype=float32)
tf.Tensor([ True  True  True  True  True  True  True  True  True  True], shape=(10,), dtype=bool)
tf.Tensor(
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(10, 1), dtype=float32)


In [18]:
result = model.predict(validation_ds_batch.take(1))

W0000 00:00:1718056952.159559 2569631 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A100-SXM4-40GB" frequency: 1410 num_cores: 108 environment { key: "architecture" value: "8.0" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 41943040 shared_memory_size_per_multiprocessor: 167936 memory_size: 742457344 bandwidth: 1555200000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2024-06-10 22:02:33.309441: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8900


1/1 [==============================] - 4s 4s/step


In [19]:
result.shape

(512, 10, 1)

In [20]:
result

array([[[0.999964  ],
        [0.9999645 ],
        [0.99996257],
        ...,
        [0.9999399 ],
        [0.99993396],
        [0.9999304 ]],

       [[0.99936086],
        [0.9991935 ],
        [0.99919504],
        ...,
        [0.9995135 ],
        [0.9995364 ],
        [0.99952304]],

       [[0.98058105],
        [0.98061013],
        [0.98071253],
        ...,
        [0.9808457 ],
        [0.980846  ],
        [0.98085004]],

       ...,

       [[0.5019611 ],
        [0.57667965],
        [0.50142384],
        ...,
        [0.43521515],
        [0.5790362 ],
        [0.6724249 ]],

       [[0.99989974],
        [0.99983716],
        [0.9998765 ],
        ...,
        [0.99986124],
        [0.9999049 ],
        [0.9998667 ]],

       [[0.32247108],
        [0.3449591 ],
        [0.3590903 ],
        ...,
        [0.32836568],
        [0.33662274],
        [0.33383223]]], dtype=float32)

In [21]:
# loss_value, accuracy = seq_model.evaluate(validation_ds_batch)
# print(f"Test accuracy: {round(accuracy * 100, 2)}%")

## Ahora generamos las predicciones

In [22]:
predictions = model.predict(validation_ds_batch)
print(predictions.shape)

34/34 [==============================] - 1s 20ms/step
(17361, 10, 1)


In [23]:
predictions[0]

array([[0.999964  ],
       [0.9999645 ],
       [0.99996257],
       [0.99996316],
       [0.99996305],
       [0.9999572 ],
       [0.99994195],
       [0.9999399 ],
       [0.99993396],
       [0.9999304 ]], dtype=float32)

In [24]:
# (173610, 7)
validation_labels = np.concatenate([labels for train_data, labels in validation_ds])
print(validation_labels.shape)

(173610, 1)


In [25]:
y_pred = np.reshape(predictions, (-1, predictions.shape[-1]))
y_pred.shape

(173610, 1)

### Ahora guardamos las predicciones

In [49]:
y_pred = np.reshape(predictions, (-1, predictions.shape[-1]))
y_pred.shape

(173610, 7)

In [26]:
np.savetxt(NUMPY_PATH + 'predictions_binary_v01.txt', y_pred, fmt='%f')
np.savetxt(NUMPY_PATH + 'validation_labels_binary_v01.txt', validation_labels, fmt='%f')

# Métricas para 7 emociones

In [3]:
y_pred = np.loadtxt(NUMPY_PATH + 'predictions_binary_v01.txt', dtype=float)
y_pred.shape

(173610,)

In [4]:
# y_pred

In [5]:
validation_labels = np.loadtxt(NUMPY_PATH + 'validation_labels_binary_v01.txt', dtype=float)
validation_labels.shape

(173610,)

In [38]:
y_true = np.reshape(validation_labels, (-1, validation_labels.shape[-1]))

# Eliminar las filas con -1 en y_true
valid_indices = np.where(np.all(y_true != -1, axis=1))
len(valid_indices[0])

0

In [35]:
# (173305, 7)
# (173305, 7)

y_true = y_true[valid_indices]
y_pred = y_pred[valid_indices]
                    
print(y_true.shape)
print(y_pred.shape)

IndexError: index 173305 is out of bounds for axis 0 with size 173305

In [29]:
y_true[0]

array([1.], dtype=float32)

In [31]:
# indices = np.where(y_true[:, 3] == 1)[0]
# indices

In [32]:
print(y_true[indices][351])
print(y_pred[indices][351])

NameError: name 'indices' is not defined

In [59]:
max_indices = tf.argmax(y_pred, axis=1)
max_indices

<tf.Tensor: shape=(173305,), dtype=int64, numpy=array([4, 4, 4, ..., 4, 4, 4])>

In [60]:
binary_pred = tf.one_hot(max_indices, depth=y_pred.shape[1])
binary_pred[0]

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 0., 0.], dtype=float32)>

# Metrics

In [61]:
def are_equal(arr1, arr2):
    return np.array_equal(arr1, arr2)

## accuracy

In [62]:
# 173305
assert len(y_true) == len(binary_pred)
print(len(y_true))

173305


In [63]:
# 95407
correct_predictions = 0
for correct, predicted in zip(y_true, binary_pred):
    if are_equal(correct, predicted):
        correct_predictions += 1
    #     print(f"equal: {correct}, {predicted}")
    # else:
    #     print(f"NOT equal: {correct}, {predicted}")
correct_predictions

90482

In [64]:
correct_predictions = sum([1 if are_equal(correct, predicted) else 0 for correct, predicted in zip(y_true, binary_pred)])
correct_predictions

90482

In [65]:
accuracy = correct_predictions / len(y_true)
f"accuracy is {accuracy:.4f}"

'accuracy is 0.5221'

## True and False positives

In [66]:
TP = np.zeros(7)
FP = np.zeros(7)
FN = np.zeros(7)

for correct, predicted in zip(y_true, binary_pred):
    correct_index = np.where(correct == 1)[0]
    predicted_index = np.where(predicted == 1)[0]
    if are_equal(correct, predicted):
        # prediction was correct
        TP[correct_index] += 1
    else:
        # prediction was incorrect
        FP[predicted_index] += 1
        FN[correct_index] += 1
print(f"TP is {TP}")
print(f"FP is {FP}")
print(f"FN is {FN}")

TP is [5.2628e+04 4.7000e+02 4.3000e+01 8.7000e+01 2.8972e+04 2.7200e+03
 5.5620e+03]
FP is [17306.  2450.  2191.  2209. 40288.  6935. 11444.]
FN is [29591.  5656.  4856.  8321.  5376. 22371.  6652.]


In [67]:
assert TP.sum() == correct_predictions

In [68]:
def get_precision(true_positives, false_positives):
    return true_positives / (true_positives + false_positives)

def get_recall(true_positives, false_negatives):
    return true_positives / (true_positives + false_negatives)

def get_f1(precision, recall):
    return 2 * precision * recall / (precision + recall)

In [69]:
F1s = []

for i, _ in enumerate(TP):
    print(f"metricas para {emociones[i]}")
    TPi = TP[i]
    FPi = FP[i]
    FNi = FN[i]
    Precisioni = get_precision(TPi, FPi)
    Recalli = get_recall(TPi, FNi)
    if Recalli != 0:
        F1i = get_f1(Precisioni, Recalli)
    else:
        F1i = 0
    F1s.append(F1i)
    print(f"Precision is {Precisioni:.4f}")
    print(f"Recalli is {Recalli:.4f}")
    print(f"F1i is {F1i:.4f}")

print(f"Macro average F1 is {(sum(F1s) / len(F1s))}")

metricas para Neutral
Precision is 0.7525
Recalli is 0.6401
F1i is 0.6918
metricas para Anger
Precision is 0.1610
Recalli is 0.0767
F1i is 0.1039
metricas para Disgust
Precision is 0.0192
Recalli is 0.0088
F1i is 0.0121
metricas para Fear
Precision is 0.0379
Recalli is 0.0103
F1i is 0.0163
metricas para Happiness
Precision is 0.4183
Recalli is 0.8435
F1i is 0.5593
metricas para Sadness
Precision is 0.2817
Recalli is 0.1084
F1i is 0.1566
metricas para Surprise
Precision is 0.3271
Recalli is 0.4554
F1i is 0.3807
Macro average F1 is 0.27436110007695125


# Métricas para la binaria

In [41]:
y_pred = np.loadtxt(NUMPY_PATH + 'predictions_binary_v01.txt', dtype=float)
y_pred = y_pred.reshape((y_pred.shape[0], 1))
# y_pred = y_pred.reshape((-1, y_pred.shape[-1]))
y_pred.shape, y_pred[0]

((173610, 1), array([0.999964]))

In [42]:
y_true = np.loadtxt(NUMPY_PATH + 'validation_labels_binary_v01.txt', dtype=float)
y_true = y_true.reshape(y_true.shape[0], 1)
# y_true = y_true.reshape(-1, y_true.shape[-1])
y_true.shape, y_true[0]

((173610, 1), array([1.]))

In [43]:
# Eliminar las filas con -1 en y_true
valid_indices = np.where(np.all(y_true != -1, axis=1))
y_true = y_true[valid_indices]
y_pred = y_pred[valid_indices]
                    
print(y_true.shape)
print(y_pred.shape)

(173305, 1)
(173305, 1)


In [44]:
np.where(y_true == 0)

(array([   198,    199,    200, ..., 167868, 167869, 167870]),
 array([0, 0, 0, ..., 0, 0, 0]))

In [45]:
y_pred[0]

array([0.999964])

In [46]:
binary_pred = (y_pred > 0.5).astype(int)

binary_pred[0]

array([1])

In [47]:
np.unique(binary_pred)

array([0, 1])

In [48]:
# Calcular la precisión
precision = tf.keras.metrics.Precision()
precision.update_state(y_true, binary_pred)
precision_result = precision.result().numpy()

print("Precisión:", precision_result)

InternalError: cudaSetDevice() on GPU:0 failed. Status: out of memory

In [ ]:
from sklearn.metrics import precision_score

precision_class_0 = precision_score(y_true, binary_pred, pos_label=0)
print(f"Precision for class Negative: {precision_class_0}")